Instalando Pacote

In [1]:
!pip install captum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.1 MB/s eta 0:00:00


In [2]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.8 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [4]:
perc= 5

In [5]:
import pandas as pd

i=1

In [6]:
arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emotion/lambada/base_emotion_'+str(perc)+'_lambada'+str(i+1)+'.csv'
lambada = pd.read_csv(arq, sep = ';', decimal = ',')

In [7]:
lambada.head()

,texto,generated,new_label
0,i miss time with my husband and not feeling ru...,i miss time with my families these days,1
1,i miss time with my husband and not feeling ru...,i miss time with my kids because you feel so w...,0
2,i miss time with my husband and not feeling ru...,i miss time with my girlfriend and im happy sh...,1
3,i miss time with my husband and not feeling ru...,i miss time with my family and friends but lat...,0
4,i miss time with my husband and not feeling ru...,i miss time with my kids but i feel really hon...,1


In [8]:
arq =  '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emotion/base_emotions_'+str(perc)+'_original'+str(i+1)+'.csv'
original = pd.read_csv(arq, sep = ';', decimal = ',')
original['generated'] = 0

In [9]:
original.head()

,texto,label,generated
0,i miss time with my husband and not feeling ru...,3,0
1,i feel tortured with tiredness everyday,4,0
2,i was feeling bouncy so i added a few of my go...,1,0
3,i feel like that s because i didn t grow up in...,1,0
4,i feel liked these days by both tom and myself,2,0


In [10]:
lambada_gen = lambada[['generated', 'new_label']].copy()
lambada_gen.rename(columns = {'generated':'texto', 'new_label':'label'}, inplace = True)
lambada_gen['generated'] = 1

lambada = original.append([lambada_gen])
lambada.head()

<ipython-input-10-fc8ec89edffa>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lambada = original.append([lambada_gen])


,texto,label,generated
0,i miss time with my husband and not feeling ru...,3,0
1,i feel tortured with tiredness everyday,4,0
2,i was feeling bouncy so i added a few of my go...,1,0
3,i feel like that s because i didn t grow up in...,1,0
4,i feel liked these days by both tom and myself,2,0


In [18]:
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab

from collections import Counter

from IPython.core.display import HTML, display

In [21]:
tokenizer = get_tokenizer('basic_english')
word_counter = Counter()
for  line in lambada['texto']:
    word_counter.update(tokenizer(line))
voc = Vocab(word_counter)

print('Vocabulary size:', len(voc))

num_class = lambada['label'].nunique()
print('Num of classes:', num_class)

Vocabulary size: 4444
Num of classes: 6


In [23]:
class EmbeddingBagModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim)
        self.linear = nn.Linear(embed_dim, num_class)

    def forward(self, inputs, offsets):
        embedded = self.embedding(inputs, offsets)
        return self.linear(embedded)

In [22]:
import numpy as np
np.random.seed(112)
df_train, df_val = np.split(lambada.sample(frac=1, random_state=42), 
                                     [int(.8*len(lambada))])

print(len(df_train),len(df_val))

3840 960


In [24]:
BATCH_SIZE = 64

def collate_batch(batch):
    labels = torch.tensor([label  for label, _ in batch]) 
    text_list = [tokenizer(line) for _, line in batch]
    
    # flatten tokens across the whole batch
    text = torch.tensor([voc[t] for tokens in text_list for t in tokens])
    # the offset of each example
    offsets = torch.tensor(
        [0] + [len(tokens) for tokens in text_list][:-1]
    ).cumsum(dim=0)

    return labels, text, offsets

train_loader = DataLoader(df_train, batch_size=BATCH_SIZE,
                          shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(df_val, batch_size=BATCH_SIZE,
                        shuffle=False, collate_fn=collate_batch)

In [27]:
EPOCHS = 7
EMB_SIZE = 64
CHECKPOINT = 'embedding_bag_ag_news.pt'
USE_PRETRAINED = False  # change to False if you want to retrain your own model

def train_model(train_loader, val_loader):
    model = EmbeddingBagModel(len(voc), EMB_SIZE, num_class)
    
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())
    
    for epoch in range(1, EPOCHS + 1):      
        # training
        model.train()
        total_acc, total_count = 0, 0
        
        for idx, (label, text, offsets) in enumerate(train_loader):
            optimizer.zero_grad()
            predited_label = model(text, offsets)
            loss(predited_label, label).backward()
            optimizer.step()
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)

            if (idx + 1) % 500 == 0:
                print('epoch {:3d} | {:5d}/{:5d} batches | accuracy {:8.3f}'.format(
                    epoch, idx + 1, len(train_loader), total_acc / total_count
                ))
                total_acc, total_count = 0, 0       
        
        # evaluation
        model.eval()
        total_acc, total_count = 0, 0

        with torch.no_grad():
            for label, text, offsets in val_loader:
                predited_label = model(text, offsets)
                total_acc += (predited_label.argmax(1) == label).sum().item()
                total_count += label.size(0)

        print('-' * 59)
        print('end of epoch {:3d} | valid accuracy {:8.3f} '.format(epoch, total_acc / total_count))
        print('-' * 59)
    
    torch.save(model, CHECKPOINT)
    return model
        
eb_model = torch.load(CHECKPOINT) if USE_PRETRAINED else train_model(train_loader, val_loader)

KeyError: ignored

In [11]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 6)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [13]:
import torch
from transformers import BertTokenizer
import numpy as np

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels =  df['label'].values
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['texto']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [16]:
np.random.seed(112)
df_train, df_val = np.split(lambada.sample(frac=1, random_state=42), 
                                     [int(.8*len(lambada))])

print(len(df_train),len(df_val))

3840 960


In [17]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 4
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 1920/1920 [06:33<00:00,  4.88it/s]


Epochs: 1 | Train Loss:  0.502                 | Train Accuracy:  0.686                 | Val Loss:  0.270                 | Val Accuracy:  0.871


 23%|██▎       | 449/1920 [01:32<05:01,  4.87it/s]


KeyboardInterrupt: ignored

In [ ]:
from captum.influence import TracInCP, TracInCPFast, TracInCPFastRandProj

In [ ]:

tracin_cp_fast = TracInCPFast(
    model=net,
    final_fc_layer=list(net.children())[-1],
    train_dataset=mislabelled_dataset,
    checkpoints=mislabelled_dataset_checkpoint_paths,
    checkpoints_load_func=checkpoints_load_func,
    loss_fn=nn.CrossEntropyLoss(reduction="sum"),
    batch_size=2048,
)

In [ ]:



arq =  '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emotion/eda/base_emotions_'+str(perc)+'_eda'+str(i+1)+'.csv'
eda = pd.read_csv(arq, sep = ';', decimal = ',')


arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emotion/back_translation/base_emotions_'+str(perc)+'_back_translation'+str(i+1)+'.csv'
back_trans = pd.read_csv(arq, sep = ';', decimal = ',')

arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emotion/teste_algo/base_emotions_'+str(perc)+'_gptneokey'+str(i+1)+'.csv'
gptneo = pd.read_csv(arq, sep = ';', decimal = ',')

arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emotion/teste_algo/base_emotions_'+str(perc)+'_gpt35key'+str(i+1)+'.csv'
gpt35 = pd.read_csv(arq, sep = ';', decimal = ',')

arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emotion/teste_algo/base_emotions_'+str(perc)+'_flant5key'+str(i+1)+'.csv'
bloom = pd.read_csv(arq, sep = ';', decimal = ',')


arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emotion/protaugment/base_emotions_'+str(perc)+'_protaugment'+str(i+1)+'.csv'
protaug = pd.read_csv(arq, sep = ';', decimal = ',')


arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emotion/lambada/base_emotion_'+str(perc)+'_lambada'+str(i+1)+'.csv'
lambada = pd.read_csv(arq, sep = ';', decimal = ',')

arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emotion/lambada/base_emotions_'+str(perc)+'_lambada_contrastive'+str(i+1)+'.csv'
c_lambada = pd.read_csv(arq, sep = ';', decimal = ',')

arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emotion/checklist/base_emotions_'+str(perc)+'_checklist'+str(i+1)+'.csv'
check = pd.read_csv(arq, sep = ';', decimal = ',')

arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emotion/ssmba/base_emotions_'+str(perc)+'_ssmba'+str(i+1)+'.csv'
ssmba = pd.read_csv(arq, sep = ';', decimal = ',')

In [ ]:
lambada.head()

,texto,generated,new_label
0,i miss time with my husband and not feeling ru...,i miss time with my families these days,1
1,i miss time with my husband and not feeling ru...,i miss time with my kids because you feel so w...,0
2,i miss time with my husband and not feeling ru...,i miss time with my girlfriend and im happy sh...,1
3,i miss time with my husband and not feeling ru...,i miss time with my family and friends but lat...,0
4,i miss time with my husband and not feeling ru...,i miss time with my kids but i feel really hon...,1


In [ ]:
bloom.head()

,texto,generated,new_label,prob_text
0,i feel irritated pissed even like when someone...,i feel irritated pissed up at these 2 slumber...,3,0.981458
1,i feel irritated pissed even like when someone...,i feel irritated and pissed when i wake up at...,3,0.981420
2,i was starting to feel resentful towards ah ki...,ive been so resentful towards ah treated to hi...,3,0.981240
3,i feel like people are aggravated with me but why,people are aggravated with me and feel like t...,3,0.981008
4,im gradually feeling a little irritated with h...,i am feeling little irritated and pacified al...,3,0.980870


In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
import sacrebleu


eda['BLEUscore'] = eda.progress_apply(lambda x: sacrebleu.sentence_bleu(x['texto'], [x['generated']]), axis=1)
back_trans['BLEUscore'] = back_trans.progress_apply(lambda x: sacrebleu.sentence_bleu(str(x['texto']), [str(x['generated'])]), axis=1)
protaug['BLEUscore'] = protaug.progress_apply(lambda x: sacrebleu.sentence_bleu(str(x['texto']), [str(x['generated'])]), axis=1)
ssmba['BLEUscore'] = ssmba.progress_apply(lambda x: sacrebleu.sentence_bleu(str(x['texto']), [str(x['generated'])]), axis=1)
check['BLEUscore'] = check.progress_apply(lambda x: sacrebleu.sentence_bleu(str(x['texto']), [str(x['generated'])]), axis=1)
lambada['BLEUscore'] = lambada.progress_apply(lambda x: sacrebleu.sentence_bleu(str(x['texto']), [str(x['generated'])]), axis=1)
c_lambada['BLEUscore'] = c_lambada.progress_apply(lambda x: sacrebleu.sentence_bleu(str(x['texto']), [str(x['generated'])]), axis=1)
bloom['BLEUscore'] = bloom.progress_apply(lambda x: sacrebleu.sentence_bleu(str(x['texto']), [str(x['generated'])]), axis=1)
gptneo['BLEUscore'] = gptneo.progress_apply(lambda x: sacrebleu.sentence_bleu(str(x['texto']), [str(x['generated'])]), axis=1)
gpt35['BLEUscore'] = gpt35.progress_apply(lambda x: sacrebleu.sentence_bleu(str(x['texto']), [str(x['generated'])]), axis=1)

100%|██████████| 4000/4000 [00:01<00:00, 2297.08it/s]


In [ ]:
eda['BLEUscore_without_bp'] = eda['BLEUscore'].progress_apply(lambda x: x.score/x.bp)
back_trans['BLEUscore_without_bp'] = back_trans['BLEUscore'].progress_apply(lambda x: x.score/x.bp)
protaug['BLEUscore_without_bp'] = protaug['BLEUscore'].progress_apply(lambda x: x.score/x.bp)
ssmba['BLEUscore_without_bp'] = ssmba['BLEUscore'].progress_apply(lambda x: x.score/x.bp)
check['BLEUscore_without_bp'] = check['BLEUscore'].progress_apply(lambda x: x.score/x.bp)
lambada['BLEUscore_without_bp'] = lambada['BLEUscore'].progress_apply(lambda x: x.score/x.bp)
c_lambada['BLEUscore_without_bp'] = c_lambada['BLEUscore'].progress_apply(lambda x: x.score/x.bp)
bloom['BLEUscore_without_bp'] = bloom['BLEUscore'].progress_apply(lambda x: x.score/x.bp)
gptneo['BLEUscore_without_bp'] = gptneo['BLEUscore'].progress_apply(lambda x: x.score/x.bp)
gpt35['BLEUscore_without_bp'] = gpt35['BLEUscore'].progress_apply(lambda x: x.score/x.bp)



100%|██████████| 4000/4000 [00:00<00:00, 421157.14it/s]


In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/paraphrase-albert-small-v2')

In [ ]:
dataset = [ eda,  back_trans, check, ssmba, protaug,lambada,c_lambada,bloom,gptneo,gpt35,]

for base in dataset:
  original_embedding = model.encode(base['texto'].astype('str'))
  base['texto_embedding'] = [original_embedding[i] for i in range(original_embedding.shape[0])]
  parafrase_embedding = model.encode(base['generated'].astype('str'))
  base['generated_embedding'] = [parafrase_embedding[i] for i in range(parafrase_embedding.shape[0])]

In [ ]:
for base in dataset:
  base['similiaridade_parafrase'] = base.progress_apply(lambda x:util.cos_sim(x['texto_embedding'], x['generated_embedding']).tolist()[0][0], axis=1)


100%|██████████| 4000/4000 [00:00<00:00, 9261.34it/s]


In [ ]:
eda.describe()

,label,BLEUscore_without_bp,similiaridade_parafrase
count,3200.000000,3200.000000,3200.000000
mean,1.568750,63.476534,0.883764
std,1.503656,15.546214,0.095851
min,0.000000,15.973578,0.380016
25%,0.000000,53.688054,0.844879
50%,1.000000,61.337060,0.913493
75%,3.000000,75.983569,0.951762
max,5.000000,100.000000,1.000000


In [ ]:
back_trans.describe()

,label,BLEUscore_without_bp,similiaridade_parafrase
count,4000.000000,4000.000000,4000.000000
mean,1.568750,26.215611,0.784966
std,1.503609,16.309148,0.133767
min,0.000000,0.000000,-0.005631
25%,0.000000,13.323584,0.714490
50%,1.000000,23.036344,0.809096
75%,3.000000,35.823421,0.886183
max,5.000000,92.539118,1.000000


In [ ]:
protaug.describe()

,label,BLEUscore_without_bp,similiaridade_parafrase
count,4000.000000,4000.000000,4000.000000
mean,1.568750,85.305223,0.956385
std,1.503609,15.887414,0.051050
min,0.000000,16.515822,0.609863
25%,0.000000,77.781112,0.945700
50%,1.000000,90.274211,0.974686
75%,3.000000,97.528956,0.987559
max,5.000000,100.000000,1.000000


In [ ]:
ssmba.describe()

,label,BLEUscore_without_bp,similiaridade_parafrase
count,4000.000000,4000.000000,4000.000000
mean,1.568750,48.894076,0.793180
std,1.503609,14.321752,0.127317
min,0.000000,7.267884,0.064721
25%,0.000000,39.832872,0.723955
50%,1.000000,49.796141,0.818438
75%,3.000000,58.772837,0.888802
max,5.000000,92.539118,0.992257


In [ ]:
check.describe()

,label,BLEUscore_without_bp,similiaridade_parafrase
count,1215.000000,1215.000000,1215.000000
mean,1.593416,93.964808,0.981450
std,1.519003,9.844903,0.052364
min,0.000000,31.947155,0.568918
25%,0.000000,89.020567,0.983115
50%,1.000000,100.000000,1.000000
75%,3.000000,100.000000,1.000000
max,5.000000,100.000000,1.000000


In [ ]:
lambada.describe()

,new_label,BLEUscore_without_bp,similiaridade_parafrase
count,4000.000000,4000.000000,4000.000000
mean,0.435000,29.986105,0.495026
std,0.495819,22.709010,0.201478
min,0.000000,3.250805,-0.086095
25%,0.000000,14.132052,0.347695
50%,0.000000,22.894157,0.487856
75%,1.000000,36.720563,0.631318
max,1.000000,100.000000,1.000000


In [ ]:
c_lambada.describe()

,new_label,BLEUscore_without_bp,similiaridade_parafrase
count,3913.000000,3913.000000,3913.000000
mean,1.721441,12.832988,0.277321
std,1.680391,11.760563,0.157342
min,0.000000,1.321984,-0.137406
25%,0.000000,5.695928,0.165109
50%,1.000000,9.103526,0.263147
75%,3.000000,14.991107,0.371476
max,5.000000,100.000000,0.958853


In [ ]:
bloom.describe()

,new_label,prob_text,BLEUscore_without_bp,similiaridade_parafrase
count,4000.000000,4000.000000,4000.000000,4000.000000
mean,1.568750,0.926660,26.464617,0.716178
std,1.503609,0.069248,22.973647,0.157111
min,0.000000,0.618274,0.000000,0.015290
25%,0.000000,0.913003,10.039524,0.619006
50%,1.000000,0.953147,18.846356,0.736789
75%,3.000000,0.969977,35.556702,0.833182
max,5.000000,0.983013,100.000000,1.000000


In [ ]:
gptneo.describe()

,new_label,prob_text,BLEUscore_without_bp,similiaridade_parafrase
count,4000.000000,4000.000000,4000.000000,4000.000000
mean,1.568750,0.962533,17.239067,0.600733
std,1.503609,0.031933,17.643067,0.166656
min,0.000000,0.747180,0.000000,-0.066645
25%,0.000000,0.962854,5.816635,0.491813
50%,1.000000,0.973679,10.693361,0.609105
75%,3.000000,0.977294,21.831744,0.724969
max,5.000000,0.980356,100.000000,0.973257


In [ ]:
gpt35.describe()

,new_label,prob_text,BLEUscore_without_bp,similiaridade_parafrase
count,4000.000000,4000.000000,4000.000000,4000.000000
mean,1.568750,0.874227,29.179051,0.688357
std,1.503609,0.152045,21.613787,0.141126
min,0.000000,0.380228,1.933389,0.174250
25%,0.000000,0.860708,12.509368,0.599573
50%,1.000000,0.945003,22.298995,0.702855
75%,3.000000,0.969073,41.113362,0.792219
max,5.000000,0.985746,100.000000,1.000000
